<a href="https://colab.research.google.com/github/ftk1000/torch_demos/blob/master/torch_gpu_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compare runtime with GPU and w/o GPU

In [1]:
import torch
import time

if torch.cuda.is_available():
  device = torch.device("cuda")  # Edit -> Notebook Settings -> Hardware Accelerator = GPU
else:
  device = torch.device("cpu")

print(device)

cuda


In [ ]:
# Get data
BATCH_SZ = 16; NUM_FEATURES = 32; HIDDEN_STATES = 128
# Note the device=cuda_device arguments here
X = torch.randn(BATCH_SZ, NUM_FEATURES,  device=device)
h = torch.randn(BATCH_SZ, HIDDEN_STATES, device=device)
C = torch.randn(BATCH_SZ, HIDDEN_STATES, device=device)